In [211]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import math
from numpy.random import random
import heapq

In [201]:
table_station = pd.read_csv('table2.csv', encoding='utf-8')
table_station = table_station.loc[:, ['CityName', 'Aws_Id', 'Lon', 'Lat']]
table_station[['x', 'y']] = table_station.apply(lambda x: millerToXY(x['Lon'], x['Lat']),
                                                axis=1, result_type="expand")
pm_data = pd.read_csv('2019aqihour.csv', encoding='gbk')
pm_data = pm_data.loc[:,['area','pm2_5','latitude','longitude','time_point']]
pm_data = pm_data[pm_data['pm2_5'].notnull()]
pm_data['time_point'] = pd.to_datetime(pm_data['time_point'], format='%Y-%m-%d %H:%M:%S')
needed_points = pd.read_csv('wind_points.csv',encoding = 'utf-8')
needed_points[['x', 'y']] = needed_points.apply(lambda x: millerToXY(x['Lon'], x['Lat']),
                                                    axis=1, result_type="expand")
needed_points = needed_points.loc[:,['SiteId','Lon','Lat','x','y']]
pm_data = pd.merge(table_station, pm_data,left_on=['CityName'], right_on=['area'],how='right')

In [242]:
table_station

,CityName,Aws_Id,Lon,Lat,x,y
0,淮北市,58015,116.33,34.45,33004036,7296086
1,宿州市,58122,116.98,33.63,33076430,7366235
2,亳州市,58102,115.77,33.87,32941665,7345747
3,阜阳市,58203,115.83,32.90,32948348,7428342
4,蚌埠市,58221,117.30,32.85,33112071,7432585
5,淮南市,58118,116.52,33.27,33025197,7396903
6,滁州市,58236,118.25,32.35,33217878,7474929
7,六安市,58311,116.50,31.73,33022970,7527237
8,合肥市,58321,117.30,31.78,33112071,7523026
9,马鞍山市,58336,118.57,31.70,33253518,7529762


In [192]:
def get_pm_data_in_timespan(pm_data,start,end):
    start = datetime.strptime(str(start), "%Y%m%d%H")
    end = datetime.strptime(str(end), "%Y%m%d%H")
    ret_data = pm_data[(pm_data['time_point']>=start) & (pm_data['time_point']<=end)]
    return ret_data

def read_data(time_start,time_end):
    day_list = get_date_list(start = time_start, end = time_end)
    all_data = []
    for d in day_list:
        file_name = 'database/'+d.strftime("%Y%m%d%H")+'.txt'
        data_of_single_day = pd.read_table(file_name,header=0,
                     usecols = ['Station_Id_d','Lat','Lon','WIN_D_Avg_2mi','WIN_S_Avg_2mi'],
                     encoding='gbk',sep='\s+')
        all_data.append(data_of_single_day[data_of_single_day['WIN_D_Avg_2mi']<=360])
    return all_data

def gen_dates(b_date, days):
    day = timedelta(days=1)
    for i in range(days):
        yield b_date + day*i
        
def gen_hours(b_hour, days, seconds):
    hours = seconds//3600
    hour = timedelta(hours=1)
    day = timedelta(days=1)
    for i in range(days):
        for j in range(24):
            yield b_hour + day*i + hour*j
    for i in range(hours):
        yield b_hour + day * days + hour*i

def get_date_list(start=None, end=None):
    """
    获取日期列表
    :param start: 开始日期 str,‘2019010100’
    :param end: 结束日期
    :return:
    """
    if start is None:
        start = datetime.strptime("2019010100", "%Y%m%d%H")
    else:
        start = datetime.strptime(str(start), "%Y%m%d%H")
    if end is None:
        end = datetime.now()
    else:
        end = datetime.strptime(str(end), "%Y%m%d%H")
    data = []
    for d in gen_dates(start, (end-start).days):
        data.append(d)
    return data

def get_hour_list(start=None, end=None):
    """
    获取日期列表
    :param start: 开始日期 str,‘2019010100’
    :param end: 结束日期
    :return:
    """
    if start is None:
        start = datetime.strptime("2019010100", "%Y%m%d%H")
    else:
        start = datetime.strptime(str(start), "%Y%m%d%H")
    if end is None:
        end = datetime.now()
    else:
        end = datetime.strptime(str(end), "%Y%m%d%H")
    data = []
    for d in gen_hours(start, (end-start).days, (end-start).seconds):
        data.append(d)
    return data

def getdistance(point, city_point):
    '''
    计算距离
    :param point:
    :param city_point:
    :return:
    '''
    return np.sqrt(np.sum((np.array(point) - np.array(city_point)) ** 2))

def millerToXY(lon, lat):
    """
    :param lon: 经度
    :param lat: 维度
    :return:
    """
    L = 6381372 * math.pi * 2  # 地球周长
    W = L  # 平面展开，将周长视为X轴
    H = L / 2  # Y轴约等于周长一般
    mill = 2.3  # 米勒投影中的一个常数，范围大约在正负2.3之间
    x = lon * math.pi / 180  # 将经度从度数转换为弧度
    y = lat * math.pi / 180
    # 将纬度从度数转换为弧度
    y = 1.25 * math.log(math.tan(0.25 * math.pi + 0.4 * y))  # 这里是米勒投影的转换

    # 这里将弧度转为实际距离 ，转换结果的单位是公里
    x = (W / 2) + (W / (2 * math.pi)) * x
    y = (H / 2) - (H / (2 * mill)) * y
    return int(round(x)), int(round(y)) 

def get_needed_points(table_origin,needed_points):
    table = pd.merge(needed_points, table_origin,
                     left_on=['SiteId'], right_on=['Station_Id_d'],how='left')
    table.drop(['Station_Id_d','Lat_y','Lon_y'],axis = 1,inplace = True)
    table = table[table.notnull()]
    return table

In [235]:
def gettime_span(pm_data,start,end):
    win_data = read_data(start,end)
    pm_data = get_pm_data_in_timespan(pm_data,start,end)
    hour_list = get_hour_list(start,end)
    pointset = sorted(set(pm_data['area'].values))
    return win_data,pm_data,hour_list,pointset

def get_edge(point_name, table, win_tables, start_time_id,needed_points,
             random_point_num=10,
             time_threshold=24, 
             distance_range = 30000):
    '''
    计算单点的单日传播图
    :param point_id:
    :param table:
    :param time:
    :param random_point_num:
    :param time_threshold:
    :return:
    '''
    point = (table[table['area'] == point_name]['x'].values[0], 
             table[table['area'] == point_name]['y'].values[0])
    # plt.scatter(table['x'],table['y'],c='b')
    # plt.scatter(point[0],point[1],c='gold')
    if len(win_tables) < start_time_id + time_threshold:
        return None,None
    
    t = random(size=random_point_num) * 2 * np.pi - np.pi
    x = np.cos(t)
    y = np.sin(t)
    length = random(size=random_point_num) * distance_range
    random_point_x = point[0] - length * x
    random_point_y = point[1] - length * y
    random_point_list = [(x, y) for x, y in zip(random_point_x, random_point_y)]
    PassingPoint = []
    PassingTime = []
    for point in random_point_list:
        # color=['g', 'r', 'c', 'm', 'y', 'k', 'w','gold','goldenrod']
        # c = np.random.choice(color)
        for i in range(time_threshold):
            win_table = win_tables[start_time_id+i]
            sub_win_table = get_needed_points(win_table,needed_points)
            
            point = movement(point, sub_win_table)
            dis = []
            for ind in range(len(table)):
                dis.append(getdistance(point, (table.iloc[ind]['x'], table.iloc[ind]['y'])))
            dis = np.array(dis)
            dis_smaller_than_thre = (dis < distance_range)
            if sum(dis_smaller_than_thre) > 0:
                # plt.scatter(point[0],point[1],c=c)
                passing_point = table.loc[dis_smaller_than_thre].loc[:, ['area']].values.flatten()

                if point_name in passing_point and len(passing_point) == 1:
                    continue
                elif point_name in passing_point:
                    passing_point = list(passing_point)
                    passing_point.remove(point_name)
                PassingPoint.extend(passing_point)
                node_time = start_time_id + i
                PassingTime.extend([node_time] * len(passing_point))
                break
    if len(PassingPoint) == 0:
        return None, None
    PassingTime = np.array(PassingTime)
    proportionDict = {}
    time_dict = {}
    for ind, key in enumerate(PassingPoint):
        proportionDict[key] = proportionDict.get(key, 0) + 1
        time_dict.setdefault(key, []).append(ind)
    # print('time_dict:',time_dict)
    for key in proportionDict.keys():
        proportionDict[key] = proportionDict[key] / len(PassingPoint)
        mean_time = cal_mean_time(PassingTime[np.array(time_dict[key])])
        time_dict[key] = mean_time
    return proportionDict, time_dict

def getdistance(point, city_point):
    '''
    计算距离
    :param point:
    :param city_point:
    :return:
    '''
    return np.sqrt(np.sum((np.array(point) - np.array(city_point)) ** 2))

def get_wind_x_y(wind_direct, wind_speed):
    '''
    将风向从角度转为 单位向量*风速
    :param wind_direct:角度
    :param wind_speed:风速
    :return:
    '''
    direct_y = math.cos(wind_direct / 180 * math.pi) * wind_speed * 3600
    direct_x = math.sin(wind_direct / 180 * math.pi) * wind_speed * 3600
    return direct_x, direct_y

def movement(point, win_table):
    '''
    计算单个点pm2.5的移动
    :param point:
    :param table:
    :return:
    '''
    dis = []
    for i in range(len(win_table)):
        city = win_table.iloc[i]
        city_point = (city['x'], city['y'])
        dis.append(getdistance(point, city_point))
    small_value_inds = list(map(dis.index, heapq.nsmallest(5, dis)))  # 求最小的五个索引    nsmallest与nlargest相反，求最小
    small_values = heapq.nsmallest(5, dis)
    small_val_neg_sum = sum([1 / v for v in small_values])
    movement_x, movement_y = 0, 0
    for ind, val in zip(small_value_inds, small_values):
        direct_x, direct_y = get_wind_x_y(win_table.iloc[ind]['WIN_D_Avg_2mi'],
                                          win_table.iloc[ind]['WIN_S_Avg_2mi'])
        movement_x += 1 / val * direct_x
        movement_y += 1 / val * direct_y
    movement_x = movement_x / small_val_neg_sum
    movement_y = movement_y / small_val_neg_sum
    point = (point[0] + movement_x, point[1] + movement_y)
    return point

def cal_mean_time(time_array):
    '''
    计算平均时间
    :param time_array:
    :return:
    '''
    time_array.sort()
    t = time_array[0] - time_array[0]
    for time in time_array:
        t = t + time - time_array[0]
    return t / len(time_array) + time_array[0]

In [233]:
win_data,pm_data,hour_list,pointset = gettime_span(pm_data,2019120100,2019123000)

In [236]:
get_edge('安庆市', pm_data, win_data, 0, needed_points,
             random_point_num=10,
             time_threshold=24, 
             distance_range = 30000)

({'安庆市': 0.8337420304070623, '池州市': 0.1662579695929377},
 {'安庆市': 0.0, '池州市': 0.0})

317.0

In [246]:
np.mean([1,2,3])

2.0

In [248]:
round(1.5)

2

In [249]:
pm_data['time_point']==''

,CityName,Aws_Id,Lon,Lat,x,y,area,pm2_5,latitude,longitude,time_point
7929,淮北市,58015,116.33,34.45,33004036,7296086,淮北市,75.0,33.940267,116.797300,2019-12-01 00:00:00
7930,淮北市,58015,116.33,34.45,33004036,7296086,淮北市,71.7,33.940267,116.797300,2019-12-01 01:00:00
7931,淮北市,58015,116.33,34.45,33004036,7296086,淮北市,69.3,33.940267,116.797300,2019-12-01 02:00:00
7932,淮北市,58015,116.33,34.45,33004036,7296086,淮北市,75.3,33.940267,116.797300,2019-12-01 03:00:00
7933,淮北市,58015,116.33,34.45,33004036,7296086,淮北市,84.7,33.940267,116.797300,2019-12-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...
497415,舟山市,58472,122.45,30.73,33685657,7611155,舟山市,36.3,29.990167,122.207133,2019-12-29 20:00:00
497416,舟山市,58472,122.45,30.73,33685657,7611155,舟山市,39.3,29.990167,122.207133,2019-12-29 21:00:00
497417,舟山市,58472,122.45,30.73,33685657,7611155,舟山市,40.3,29.990167,122.207133,2019-12-29 22:00:00
497418,舟山市,58472,122.45,30.73,33685657,7611155,舟山市,41.7,29.990167,122.207133,2019-12-29 23:00:00


In [188]:
a = get_needed_points(data[0],needed_points)

In [191]:
sorted(a['WIN_D_Avg_2mi'])

[3.0,
 3.0,
 4.0,
 4.0,
 5.0,
 6.0,
 6.0,
 6.0,
 8.0,
 9.0,
 10.0,
 12.0,
 15.0,
 23.0,
 23.0,
 27.0,
 55.0,
 65.0,
 71.0,
 249.0,
 254.0,
 254.0,
 256.0,
 256.0,
 257.0,
 261.0,
 267.0,
 267.0,
 269.0,
 273.0,
 273.0,
 273.0,
 274.0,
 276.0,
 276.0,
 276.0,
 276.0,
 277.0,
 278.0,
 278.0,
 279.0,
 280.0,
 281.0,
 281.0,
 282.0,
 282.0,
 284.0,
 284.0,
 285.0,
 286.0,
 286.0,
 287.0,
 287.0,
 287.0,
 288.0,
 288.0,
 288.0,
 289.0,
 290.0,
 291.0,
 291.0,
 292.0,
 293.0,
 293.0,
 293.0,
 294.0,
 294.0,
 294.0,
 295.0,
 295.0,
 295.0,
 295.0,
 295.0,
 297.0,
 299.0,
 300.0,
 300.0,
 301.0,
 301.0,
 302.0,
 304.0,
 304.0,
 305.0,
 305.0,
 305.0,
 306.0,
 306.0,
 308.0,
 308.0,
 309.0,
 310.0,
 311.0,
 311.0,
 313.0,
 315.0,
 315.0,
 316.0,
 316.0,
 318.0,
 319.0,
 319.0,
 320.0,
 322.0,
 324.0,
 327.0,
 329.0,
 329.0,
 330.0,
 333.0,
 337.0,
 339.0,
 341.0,
 341.0,
 342.0,
 342.0,
 345.0,
 347.0,
 347.0,
 349.0,
 352.0,
 355.0,
 355.0,
 356.0,
 357.0,
 357.0,
 359.0,
 359.0,
 360.0,
 nan,

In [251]:
'2019-12-30 00:00:01'>'2019-12-30 00:00:00'

True

In [7]:
import numpy as np
a = np.zeros([10,5])

In [40]:
a= np.load('edge_Pij_dict.npy',allow_pickle=True).item()
a

{0: [['南京市', '马鞍山市', 1.0, 0, 1, 41.0],
  ['安庆市', '池州市', 1.0, 0, 2, 35.3],
  ['宣城市', '黄山市', 0.75, 0, 10, 45.0],
  ['宿州市', '淮南市', 1.0, 0, 3, 46.7],
  ['常州市', '芜湖市', 0.8571428571428571, 0, 10, 52.54285714285714],
  ['徐州市', '宿州市', 1.0, 0, 4, 48.8],
  ['扬州市', '南京市', 1.0, 0, 5, 45.5],
  ['池州市', '安庆市', 1.0, 0, 0, 40.0],
  ['泰州市', '常州市', 0.9, 0, 5, 35.28],
  ['淮北市', '亳州市', 1.0, 0, 6, 75.0],
  ['淮南市', '阜阳市', 1.0, 0, 6, 69.4],
  ['淮安市', '滁州市', 1.0, 0, 6, 75.0],
  ['滁州市', '合肥市', 1.0, 0, 6, 56.0],
  ['芜湖市', '铜陵市', 0.9, 0, 4, 39.87],
  ['蚌埠市', '六安市', 1.0, 0, 6, 60.5],
  ['镇江市', '南京市', 1.0, 0, 3, 48.8],
  ['马鞍山市', '铜陵市', 0.8333333333333334, 0, 6, 39.583333333333336]],
 1: [['南京市', '马鞍山市', 1.0, 1, 3, 34.9],
  ['安庆市', '池州市', 1.0, 1, 2, 36.3],
  ['宣城市', '黄山市', 0.6666666666666666, 1, 9, 40.0],
  ['宿州市', '淮南市', 1.0, 1, 3, 60.7],
  ['常州市', '芜湖市', 1.0, 1, 13, 61.8],
  ['徐州市', '宿州市', 1.0, 1, 5, 51.8],
  ['泰州市', '常州市', 0.8, 1, 7, 32.0],
  ['淮北市', '亳州市', 1.0, 1, 7, 71.7],
  ['淮南市', '阜阳市', 1.0, 1, 9, 69.0],
  

In [32]:
from collections import defaultdict
edge = defaultdict(list)
edge2 = defaultdict(int)

In [39]:
a = [1,2]
a.append([1,2,3])
a

[1, 2, [1, 2, 3]]

In [33]:
edge2[(1,2)].append(1)

AttributeError: 'int' object has no attribute 'append'

In [2]:
from pyecharts.charts import Bar
from pyecharts.charts import Geo
from pyecharts.charts import Map

In [8]:
from pyecharts.charts import Map, Geo
# 世界地图数据
value = [95.1, 23.2, 43.3, 66.4, 88.5]
attr= ["China", "Canada", "Brazil", "Russia", "United States"]

# 省和直辖市
province_distribution = {'河南': 45.23, '北京': 37.56, '河北': 21, '辽宁': 12, '江西': 6, '上海': 20, '安徽': 10, '江苏': 16, '湖南': 9, '浙江': 13, '海南': 2, '广东': 22, '湖北': 8, '黑龙江': 11, '澳门': 1, '陕西': 11, '四川': 7, '内蒙古': 3, '重庆': 3, '云南': 6, '贵州': 2, '吉林': 3, '山西': 12, '山东': 11, '福建': 4, '青海': 1, '舵主科技，质量保证': 1, '天津': 1, '其他': 1}
provice=list(province_distribution.keys())
values=list(province_distribution.values())

# 城市 -- 指定省的城市 xx市
city = ['郑州市', '安阳市', '洛阳市', '濮阳市', '南阳市', '开封市', '商丘市', '信阳市', '新乡市']
values2 = [1.07, 3.85, 6.38, 8.21, 2.53, 4.37, 9.38, 4.29, 6.1]

# 区县 -- 具体城市内的区县  xx县
quxian = ['夏邑县', '民权县', '梁园区', '睢阳区', '柘城县', '宁陵县']
values3 = [3, 5, 7, 8, 2, 4]

In [10]:
map0 = Map("世界地图示例")
map0.add("世界地图", attr, value, maptype="world",  is_visualmap=True, visual_text_color='#000')
map0.render(path="./data/04-00世界地图.html")

AttributeError: 'str' object has no attribute 'get'

In [5]:
from pyecharts.charts import Map, Geo
# 世界地图数据
value = [95.1, 23.2, 43.3, 66.4, 88.5]
attr= ["China", "Canada", "Brazil", "Russia", "United States"]

# 省和直辖市
province_distribution = {'河南': 45.23, '北京': 37.56, '河北': 21, '辽宁': 12, '江西': 6, '上海': 20, '安徽': 10, '江苏': 16, '湖南': 9, '浙江': 13, '海南': 2, '广东': 22, '湖北': 8, '黑龙江': 11, '澳门': 1, '陕西': 11, '四川': 7, '内蒙古': 3, '重庆': 3, '云南': 6, '贵州': 2, '吉林': 3, '山西': 12, '山东': 11, '福建': 4, '青海': 1, '舵主科技，质量保证': 1, '天津': 1, '其他': 1}
provice=list(province_distribution.keys())
values=list(province_distribution.values())

# 城市 -- 指定省的城市 xx市
city = ['郑州市', '安阳市', '洛阳市', '濮阳市', '南阳市', '开封市', '商丘市', '信阳市', '新乡市']
values2 = [1.07, 3.85, 6.38, 8.21, 2.53, 4.37, 9.38, 4.29, 6.1]

# 区县 -- 具体城市内的区县  xx县
quxian = ['夏邑县', '民权县', '梁园区', '睢阳区', '柘城县', '宁陵县']
values3 = [3, 5, 7, 8, 2, 4]
map0 = Map("世界地图示例")
map0.add("世界地图", attr, value, maptype="world",  is_visualmap=True, visual_text_color='#000')
map0.render(path="04-00世界地图.html")

AttributeError: 'str' object has no attribute 'get'

In [8]:
# maptype='china' 只显示全国直辖市和省级
# 数据只能是省名和直辖市的名称
map = Map("中国地图",'中国地图')
map.add("", provice, values, visual_range=[0, 50],  maptype='china', is_visualmap=True,
    visual_text_color='#000')
#map.show_config()
map.load_javascript()
map.render_notebook()

TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given

In [11]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType


# 链式调用
c = (
    Geo()
    # 加载图表模型中的中国地图
    .add_schema(maptype="china")
    # 在地图中加入点的属性
    .add("geo", [list(z) for z in zip(Faker.provinces, Faker.values())])
    # 设置坐标属性
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    # 设置全局属性
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(),
        title_opts=opts.TitleOpts(title="Geo-基本示例"),
    )
)

# 在 html(浏览器) 中渲染图表
c.render()
# 在 Jupyter Notebook 中渲染图表
c.render_notebook()


/opt/anaconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [1]:
from build_graph_hourly import BuildGraphHourly
bgh = BuildGraphHourly(start="2019120100", end="2019120103",is_build_graph=False)
bgh.getGraphList(reload='edge_Pij_dict.npy')

In [18]:
for key, graph in bgh.graphdict.items():
    if graph.get_vertex_num() < 3:
        continue
    print(key)

11
25
27
35
38


In [22]:
bgh.graphdict[11].getGraph()

gid: 11
edge: defaultdict(<class 'list'>, {(1, 9): [('马鞍山市', '铜陵市')], (0, 1): [('南京市', '马鞍山市')]})
vertices: defaultdict(<class 'list'>, {9: ['铜陵市'], 1: ['马鞍山市'], 0: ['南京市']})
-------------------


In [23]:
bgh.graphdict[38].getGraph()

gid: 38
edge: defaultdict(<class 'list'>, {(0, 2): [('安庆市', '池州市')], (1, 2): [('安庆市', '池州市')]})
vertices: defaultdict(<class 'list'>, {2: ['池州市'], 0: ['安庆市'], 1: ['安庆市']})
-------------------


In [2]:
bgh.name_xy_dict

{'上海市': (33574281, 7554990),
 '丽水市': (33403876, 7800527),
 '亳州市': (32941665, 7345747),
 '六安市': (33022970, 7527237),
 '南京市': (33291386, 7510387),
 '南通市': (33521934, 7497735),
 '台州市': (33492976, 7765839),
 '合肥市': (33112071, 7523026),
 '嘉兴市': (33537527, 7617845),
 '宁波市': (33585418, 7690382),
 '安庆市': (33084227, 7627874),
 '宣城市': (33299182, 7620353),
 '宿州市': (33076430, 7366235),
 '宿迁市': (33276907, 7327790),
 '常州市': (33410558, 7514601),
 '徐州市': (33095364, 7310663),
 '扬州市': (33351529, 7436826),
 '无锡市': (33451767, 7536495),
 '杭州市': (33379373, 7653746),
 '池州市': (33080885, 7663748),
 '泰州市': (33476270, 7473238),
 '淮北市': (33004036, 7296086),
 '淮南市': (33025197, 7396903),
 '淮安市': (33293613, 7366235),
 '温州市': (33485180, 7855686),
 '湖州市': (33417241, 7599439),
 '滁州市': (33217878, 7474929),
 '盐城市': (33466246, 7402857),
 '绍兴市': (33504114, 7667913),
 '舟山市': (33685657, 7611155),
 '芜湖市': (33254632, 7575976),
 '苏州市': (33460677, 7582684),
 '蚌埠市': (33112071, 7432585),
 '衢州市': (33290272, 7755085),
 '连云港市': (3332

In [3]:
from pyecharts import GeoLines, Style

ImportError: cannot import name 'GeoLines' from 'pyecharts' (/opt/anaconda3/lib/python3.7/site-packages/pyecharts/__init__.py)

In [1]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType

c = (
    Geo()
    .add_schema(maptype="china")
    .add(
        "",
        [("马鞍山市", 55), ("铜陵市", 66), ("南京市", 77)],
        type_=ChartType.EFFECT_SCATTER,
        color="white",
        label_opts=opts.LabelOpts(
            # 是否显示标签。
            is_show = True,
            position = "top",
        )
    )
    .add(
        "geo",
        [("马鞍山市", "铜陵市"), ('南京市', '马鞍山市')],
        type_=ChartType.LINES,
        effect_opts=opts.EffectOpts(
            symbol=SymbolType.ARROW, symbol_size=6, color="blue",
        ),
        linestyle_opts=opts.LineStyleOpts(curve=0.2),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Geo-Lines"),
        visualmap_opts=opts.VisualMapOpts(max_=500),
    )
    .render()
)


/opt/anaconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [18]:
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.faker import Faker

c = (
    Map()
    .add("商家A", [list(z) for z in zip(Faker.provinces, Faker.values())], "china")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Map-VisualMap（连续型）"),
        visualmap_opts=opts.VisualMapOpts(max_=200),
    )
    .render("test.html")
)


/opt/anaconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
